<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/multi_class_using_Gemini_zero_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [1]:
!pip install --upgrade google-cloud-aiplatform

2. Use the following code in your application to request a model response

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/keyfile.json" # replace with the actual path to your key file

In [2]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting, Part

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]

def multiturn_generate_content_techniques(id, input_text):
    vertexai.init(project="spry-semiotics-251121", location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-flash-002",
    )
    common_prompt = """please identify multiple classes with probabilities from the list ("Supervised Machine Learning", "Unsupervised Machine Learning", "Reinforced Learning", "Classification / Clustering Techniques", "Decision Trees", "Regression",
"Time Series", "Deep Learning", "NLP", "NLG", "GenAI", "Computer Vision / Image/Video Recognition", "Audio Analysis / Signal Processing / Speech Recognition", "Recommendation System"] for the below description in JSON output including ID passed in.
 """ + " ID: """ + str(id) + """Description: """
    prompt_text = common_prompt + input_text
    chat = model.start_chat()
    return(chat.send_message(
        [prompt_text],
        generation_config=generation_config,
        safety_settings=safety_settings
    ))

text1_1 = """Our Capstone project focuses on predicting arbitrage opportunities in time series liquidity pool transactions. Our tool enables cryptocurrency traders to make smarter decisions, optimize their strategies, and increase returns. By providing accurate predictions, traders can better navigate the volatile crypto market and maximize their profitability. We analyze transaction data from Ethereum-based liquidity pools using advanced machine learning models, including an LGBM model and XG Boost. Our approach involves predicting percent differences in currencies, incorporating external factors like global market prices and transaction fees to enhance prediction accuracy. We seek support through access to more data sources, mentorship from industry experts, and partnerships with crypto trading platforms to refine and validate our tool. Your collaboration will help us bring this innovative solution to the market and revolutionize how traders engage with liquidity pools.Dataset and ModelingArbitrage Equation: Net Gain = A(1+ΔP)(1-T2) - A(1-T2) - GA is the amount investedA(1+ΔP) is the value obtained after arbitrageT1 and T2 are the transaction fees (constant, based on the pool providers)G is the gas fees (computational cost of doing a transaction on the blockchain)The type of arbitrage that we are looking at is where we buy a token at a lower price in one pool and sell it for a higher price in the other pool. To have a net gain we first need an amount that we are willing to invest. Theoretically we will be making this amount back and capturing the difference in the pricing between the two pools. So A times the 1 plus the change in percentage will be the value we obtain after the arbitrage opportunity. Next we need to subtract the transaction fees that are incurred for trading in the two pools as well as the Gas fees. This results in us getting the Net Gain. To get the minimum amount we need to invest we set our net gain equal to zero and solve for A. Minimum Amount to Invest:A = G/((1+ΔP)(1-T2) - (1-T2))Here we see that gas prices and the percent change are the only variables that vary per transaction and these will be what we predict in our modeling.Our data was sourced from Uniswap V3 data that was hosted on the Graph. The data was collated by querying the Graph API for two specific WETH/USDC liquidity pools. The queries returned transactional data from each of the pools which we needed to process and marry based on the timestamps of each transaction. As such these pools needed to have a relatively high volume of transactions to ensure that we get transactions that were close to one another. Once records of transactions that occurred close enough to one another were obtained we could calculate the percent difference between the prices of the WETH coin in both liquidity pools as well as the total gas fees from each transaction.In our modeling approach we experimented with a variety of ways to put our data together. Initially we had went to source various data types that we thought would help in predicting a large differences in prices between two pools. However, through our eda and modeling we quickly discovered that the other features were clouding the model’s judgement and creating a lot of noise. The features that helped our model performance the most were the various lags, means moving averages of the target variable. In our initial approach where we just slid our target variable back by 5 minutes. Our random forest, linear regression and xgb boosts used this approach. In our LGBM model we experimented with a different approach. First we took the mean at each minute interval. Then we lagged the mean and then slid our target variable back by 5 minutes. This helped the performance of our LGBM by smoothing the data that it was predicting. For our LSTM model, we followed a windowing approach. We start with our target variable and find the entry that is exactly 5 minutes prior. Then we take the lags at a set frequency to make up the entries that will help predict our target variable. When you stack these you then get a dataset. We experimented with different frequencies of the lags and the amount of time before prior to the target variable.  EvaluationTo predict the percent change, we explored various models. We scored our models based on the RMSE (Root Mean Squared Error) and the R2 (wellness of fit). TypeRMSE (% Change)R2Random Forest0.001270.375XGBoost0.001340.421Linear (Baseline)0.001050.573LSTM0.0009630.598LGBM0.0009490.646Our baseline was the linear regression model which performed surprisingly well. We tuned the hyperparameters of our better performing models to ensure we were getting the best performance from these. We explored tree based models like random forest, xgboost and LGBMs. The LGBM was by far the better performing tree based model. This makes sense as in its approach it grows the leafs that maximize the loss, which in turn allows this model to predict more accurately than a random forest or xgboost which grows the trees level wise. The LSTM was a close second and it’s architecture of windows works rather well at time series problems. Next we predicted the gas prices. The gas price is the computational cost of doing a transaction on the blockchain. We observed that the actual gas had sudden spikes all throughout the days. This could be due to various factors such as spikes in demand, miner behavior, network congestion in the blockchain and so on. However, when the gas prices are relatively constant, we see that the models are performing well. Due to the spikes, we saw the R2 suffer across the board.TypeRMSE (total gas price)R2Linear (Baseline)48613-0.015LSTM145.170.112LGBM266.390.170XGBoost266.360.171We kept our linear regression as our baseline. We had to retune the hyperparameters for these models to be able to predict the gas fees. Our best performing model was the XGboost model, we see it outperforming the LGBM this time around. This is likely due to the LGBM overfitting the training data and we observed cases of this where the model predicted consistently higher than the actual gas fee and there were also instances where it attempts to predict these spikes.Impact and Technical ChallengesPutting the predictions of the gas fees and the percentage change together we are able to find the minimum amount that we should invest. Using the actual percent change and gas fees during that time we are able to see what our returns. Our strategy here is the invest at every opportunity where the model predicts a positive amount that is within a specified budget. From our test runs we observed what our returns would be using this investment strategy at different budgets. We saw that at a budget of 6000 if we invest using this strategy we can make a return of 8000. However on the flip side if we had a high enough budget around 10,000 dollars we could potentially lose over 3 million dollars. This is a trend that we have seen consistently that at higher amount of investments there is a potentially large downside. Our deep dive into the world of decentralized finance faced 3 major technical challengesFirstly, identifying and executing arbitrage opportunities for profit consistently is really difficult. we\'re dealing with an incredibly challenging arena. Cross-liquidity pool arbitrage opportunities are scarce, as the market is saturated with professional institutions who have significant advantages in terms of computing power, speed and investment size. Moreover, predicting slippage and gas prices accurately is also very difficult, making it difficult to gauge true profitability.The second challenge revolves around the complexity of blockchain data. While it is all open and freely available, parsing raw blockchain data efficiently without expensive premium tools isn’t easy - there are a lot of data startups in the space working to solve for this but their premium tools are expensive, putting retail traders at a disadvantage when trying to identify opportunities quickly.Lastly, we must contend with Ethereum\'s \'dark forest\' nature. All transactions made on the blockchain are visible in the mempool before confirmation, leaving them exposed. This means anyone can potentially front-run your swap, trade or order, and in our case stealing our arbitrage opportunity. MEV is basically when clever traders (or even more clever bots and algorithms) make money by frontrunning you in the split second before your crypto trade goes through. The presence of these types of bots make it hard to compete without institutional grade hardware and data"""


y = multiturn_generate_content_techniques(123, text1_1)
y

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7d8174828e50>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Go

ServiceUnavailable: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7d8174517460>)

In [ ]:
def multiturn_generate_content_topics(id, input_text):
    vertexai.init(project="spry-semiotics-251121", location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-flash-002",
    )
    common_prompt = """please identify the most probable class from the list ("Supervised Machine Learning", "Unsupervised Machine Learning", "Reinforced Learning", "Classification / Clustering Techniques", "Decision Trees", "Regression",
"Time Series", "Deep Learning", "NLP", "NLG", "GenAI", "Computer Vision / Image/Video Recognition", "Audio Analysis / Signal Processing / Speech Recognition", "Recommendation System"] for the below description in JSON output including ID passed in.
 """ + " ID: """ + str(id) + """Description: """
    prompt_text = common_prompt + input_text
    chat = model.start_chat()
    return(chat.send_message(
        [prompt_text],
        generation_config=generation_config,
        safety_settings=safety_settings
    ))

text1_1 = """Our Capstone project focuses on predicting arbitrage opportunities in time series liquidity pool transactions. Our tool enables cryptocurrency traders to make smarter decisions, optimize their strategies, and increase returns. By providing accurate predictions, traders can better navigate the volatile crypto market and maximize their profitability. We analyze transaction data from Ethereum-based liquidity pools using advanced machine learning models, including an LGBM model and XG Boost. Our approach involves predicting percent differences in currencies, incorporating external factors like global market prices and transaction fees to enhance prediction accuracy. We seek support through access to more data sources, mentorship from industry experts, and partnerships with crypto trading platforms to refine and validate our tool. Your collaboration will help us bring this innovative solution to the market and revolutionize how traders engage with liquidity pools.Dataset and ModelingArbitrage Equation: Net Gain = A(1+ΔP)(1-T2) - A(1-T2) - GA is the amount investedA(1+ΔP) is the value obtained after arbitrageT1 and T2 are the transaction fees (constant, based on the pool providers)G is the gas fees (computational cost of doing a transaction on the blockchain)The type of arbitrage that we are looking at is where we buy a token at a lower price in one pool and sell it for a higher price in the other pool. To have a net gain we first need an amount that we are willing to invest. Theoretically we will be making this amount back and capturing the difference in the pricing between the two pools. So A times the 1 plus the change in percentage will be the value we obtain after the arbitrage opportunity. Next we need to subtract the transaction fees that are incurred for trading in the two pools as well as the Gas fees. This results in us getting the Net Gain. To get the minimum amount we need to invest we set our net gain equal to zero and solve for A. Minimum Amount to Invest:A = G/((1+ΔP)(1-T2) - (1-T2))Here we see that gas prices and the percent change are the only variables that vary per transaction and these will be what we predict in our modeling.Our data was sourced from Uniswap V3 data that was hosted on the Graph. The data was collated by querying the Graph API for two specific WETH/USDC liquidity pools. The queries returned transactional data from each of the pools which we needed to process and marry based on the timestamps of each transaction. As such these pools needed to have a relatively high volume of transactions to ensure that we get transactions that were close to one another. Once records of transactions that occurred close enough to one another were obtained we could calculate the percent difference between the prices of the WETH coin in both liquidity pools as well as the total gas fees from each transaction.In our modeling approach we experimented with a variety of ways to put our data together. Initially we had went to source various data types that we thought would help in predicting a large differences in prices between two pools. However, through our eda and modeling we quickly discovered that the other features were clouding the model’s judgement and creating a lot of noise. The features that helped our model performance the most were the various lags, means moving averages of the target variable. In our initial approach where we just slid our target variable back by 5 minutes. Our random forest, linear regression and xgb boosts used this approach. In our LGBM model we experimented with a different approach. First we took the mean at each minute interval. Then we lagged the mean and then slid our target variable back by 5 minutes. This helped the performance of our LGBM by smoothing the data that it was predicting. For our LSTM model, we followed a windowing approach. We start with our target variable and find the entry that is exactly 5 minutes prior. Then we take the lags at a set frequency to make up the entries that will help predict our target variable. When you stack these you then get a dataset. We experimented with different frequencies of the lags and the amount of time before prior to the target variable.  EvaluationTo predict the percent change, we explored various models. We scored our models based on the RMSE (Root Mean Squared Error) and the R2 (wellness of fit). TypeRMSE (% Change)R2Random Forest0.001270.375XGBoost0.001340.421Linear (Baseline)0.001050.573LSTM0.0009630.598LGBM0.0009490.646Our baseline was the linear regression model which performed surprisingly well. We tuned the hyperparameters of our better performing models to ensure we were getting the best performance from these. We explored tree based models like random forest, xgboost and LGBMs. The LGBM was by far the better performing tree based model. This makes sense as in its approach it grows the leafs that maximize the loss, which in turn allows this model to predict more accurately than a random forest or xgboost which grows the trees level wise. The LSTM was a close second and it’s architecture of windows works rather well at time series problems. Next we predicted the gas prices. The gas price is the computational cost of doing a transaction on the blockchain. We observed that the actual gas had sudden spikes all throughout the days. This could be due to various factors such as spikes in demand, miner behavior, network congestion in the blockchain and so on. However, when the gas prices are relatively constant, we see that the models are performing well. Due to the spikes, we saw the R2 suffer across the board.TypeRMSE (total gas price)R2Linear (Baseline)48613-0.015LSTM145.170.112LGBM266.390.170XGBoost266.360.171We kept our linear regression as our baseline. We had to retune the hyperparameters for these models to be able to predict the gas fees. Our best performing model was the XGboost model, we see it outperforming the LGBM this time around. This is likely due to the LGBM overfitting the training data and we observed cases of this where the model predicted consistently higher than the actual gas fee and there were also instances where it attempts to predict these spikes.Impact and Technical ChallengesPutting the predictions of the gas fees and the percentage change together we are able to find the minimum amount that we should invest. Using the actual percent change and gas fees during that time we are able to see what our returns. Our strategy here is the invest at every opportunity where the model predicts a positive amount that is within a specified budget. From our test runs we observed what our returns would be using this investment strategy at different budgets. We saw that at a budget of 6000 if we invest using this strategy we can make a return of 8000. However on the flip side if we had a high enough budget around 10,000 dollars we could potentially lose over 3 million dollars. This is a trend that we have seen consistently that at higher amount of investments there is a potentially large downside. Our deep dive into the world of decentralized finance faced 3 major technical challengesFirstly, identifying and executing arbitrage opportunities for profit consistently is really difficult. we\'re dealing with an incredibly challenging arena. Cross-liquidity pool arbitrage opportunities are scarce, as the market is saturated with professional institutions who have significant advantages in terms of computing power, speed and investment size. Moreover, predicting slippage and gas prices accurately is also very difficult, making it difficult to gauge true profitability.The second challenge revolves around the complexity of blockchain data. While it is all open and freely available, parsing raw blockchain data efficiently without expensive premium tools isn’t easy - there are a lot of data startups in the space working to solve for this but their premium tools are expensive, putting retail traders at a disadvantage when trying to identify opportunities quickly.Lastly, we must contend with Ethereum\'s \'dark forest\' nature. All transactions made on the blockchain are visible in the mempool before confirmation, leaving them exposed. This means anyone can potentially front-run your swap, trade or order, and in our case stealing our arbitrage opportunity. MEV is basically when clever traders (or even more clever bots and algorithms) make money by frontrunning you in the split second before your crypto trade goes through. The presence of these types of bots make it hard to compete without institutional grade hardware and data"""


y = multiturn_generate_content_techniques(123, text1_1)
y

In [ ]:
import pandas as pd
data = pd.read_csv("gs://prabhutest/mids-final-projects - mids-final-projects.csv")
data

,ID,URL,Capstone Category,Project Term,title,Blurb,Description
0,19121,https://www.ischool.berkeley.edu/node/19121,MIDS Capstone Project,Summer 2024,Kairos,Kairós is a web app that uses machine learning...,Problem & MotivationWith the growing complexit...
1,19120,https://www.ischool.berkeley.edu/node/19120,MIDS Capstone Project,Summer 2024,Liquidity Pool Arbitrage,Predicting arbitrage opportunities in Ethereum...,DescriptionOur Capstone project focuses on pre...
2,19119,https://www.ischool.berkeley.edu/node/19119,MIDS Capstone Project,Summer 2024,GabbleGrid: Self Healing Clouds with AI Agents,GabbleGrid transforms cloud service management...,Developed as a UC Berkeley graduate capstone i...
3,19118,https://www.ischool.berkeley.edu/node/19118,MIDS Capstone Project,Summer 2024,RECO: Recovery Companion,A generative AI-powered chatbot to reduce hear...,Problem and MotivationOne in 33 Americans suff...
4,19117,https://www.ischool.berkeley.edu/node/19117,MIDS Capstone Project,Summer 2024,Carbon AIQ,Empowering individuals and organizations with ...,Empowering individuals and organizations with ...
...,...,...,...,...,...,...,...
574,4607,https://www.ischool.berkeley.edu/node/4607,MIDS Capstone Project,Summer 2015,Wilson,Discover which Lending Club neighbors to inves...,Wilson is a P2P Loan recommendation tool that ...
575,4600,https://www.ischool.berkeley.edu/node/4600,MIDS Capstone Project,Summer 2015,Predicting the Next Pitch,Predicting the next pitch in a baseball game u...,This is the landing page for the Pitch Predict...
576,4613,https://www.ischool.berkeley.edu/node/4613,MIDS Capstone Project,Summer 2015,R.A.P. - Rap Analysis Project,"A data science exploration of rap lyrics, and ...",As a capstone project for the MIDS program at ...
577,4612,https://www.ischool.berkeley.edu/node/4612,MIDS Capstone Project,Summer 2015,Where should you live?,Enabling data-driven home buying decisions,Buying a home requires a lot of important deci...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 579 non-null    int64 
 1   URL                579 non-null    object
 2   Capstone Category  579 non-null    object
 3   Project Term       579 non-null    object
 4   title              579 non-null    object
 5   Blurb              535 non-null    object
 6   Description        576 non-null    object
dtypes: int64(1), object(6)
memory usage: 31.8+ KB


In [ ]:
# prompt: write code to iterate through data pandas dataframe and call function "multiturn_generate_content" with passing column "description" as a parameter and add results and ID column to array. At the end convert Array to dataframe

results = []
# i = 0
for index, row in data.iterrows():
  try:
    if row['Description'] == "":
      continue
    response = multiturn_generate_content(row['ID'], row['Description'])
    results.append(response)

  except TypeError as e:
      print ("ID: ", row['ID'], "\n Description: " , row['Description'] , "\n")
      print (e)
      continue
  # if i == 15: break
  # i += 1

# results


ID:  17493 
 Description:  nan 

can only concatenate str (not "float") to str
ID:  10468 
 Description:  nan 

can only concatenate str (not "float") to str
ID:  10274 
 Description:  nan 

can only concatenate str (not "float") to str


In [ ]:
# results

In [ ]:
# prompt: generate code to parts element in the below array of jsons and flatten classes to ID level
# [candidates {
#    content {
#      role: "model"
#      parts {
#        text: "```json\n{\n  \"ID\": 19121,\n  \"classes\": [\n    {\n      \"class\": \"Supervised Machine Learning\",\n      \"probability\": 0.95\n    },\n    {\n      \"class\": \"Classification / Clustering Techniques\",\n      \"probability\": 0.90\n    },\n    {\n      \"class\": \"Regression\",\n      \"probability\": 0.70 \n    },\n    {\n      \"class\": \"Deep Learning\",\n      \"probability\": 0.85\n    },\n    {\n      \"class\": \"Decision Trees\",\n      \"probability\": 0.75\n    },\n    {\n      \"class\": \"Reinforced Learning\",\n      \"probability\": 0.20 \n    },\n    {\n      \"class\": \"Unsupervised Machine Learning\",\n      \"probability\": 0.10\n    },\n   ]

import json
import pandas as pd

def process_response(response):
  try:
    content = response.text
    json_str = content.split("```json")[1].split("```")[0].strip()
    data = json.loads(json_str)
    classes = data.get('classes', [])
    flattened_classes = {f"{c['class']} Probability": c['probability'] for c in classes}
    data.update(flattened_classes)
    del data['classes']
    return data
  except Exception as e:
    print(f"Error processing response: {e}, {response.text}")
    return {}

processed_results = [process_response(r) for r in results]
df_results = pd.DataFrame(processed_results)
df_results

Error processing response: 'classes', ```json
{
  "ID": 19108,
  "Classes": [
    {
      "class": "Regression",
      "probability": 0.95
    },
    {
      "class": "Supervised Machine Learning",
      "probability": 0.90
    },
    {
      "class": "Deep Learning",
      "probability": 0.70 
    },
    {
      "class": "Classification / Clustering Techniques",
      "probability": 0.60
    },
    {
      "class": "Decision Trees",
      "probability": 0.55
    },
    {
      "class": "Time Series",
      "probability": 0.40
    },
    {
      "class": "Computer Vision / Image/Video Recognition",
      "probability": 0.10
    },
    {
      "class": "Recommendation System",
      "probability": 0.05
    },
    {
      "class": "Unsupervised Machine Learning",
      "probability": 0.05
    },
    {
      "class": "Reinforced Learning",
      "probability": 0.05
    },
    {
      "class": "NLP",
      "probability": 0.05
    },
    {
      "class": "NLG",
      "probability": 0.05
   

,ID,Supervised Machine Learning Probability,Classification / Clustering Techniques Probability,Regression Probability,Deep Learning Probability,Decision Trees Probability,Reinforced Learning Probability,Unsupervised Machine Learning Probability,Time Series Probability,NLP Probability,...,Data Visualization Probability,Prediction Probability,Hedonic Modeling Probability,Convolutional Neural Networks Probability,Conservation Probability,Anomaly Detection Probability,Big Data Probability,Data Mining Probability,MapReduce Probability,Machine Learning Probability
0,19121.0,0.95,0.9,0.70,0.85,0.75,0.10,0.2,0.10,0.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19120.0,0.90,0.7,0.85,0.75,0.65,0.50,0.1,0.95,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19119.0,0.80,0.9,0.50,0.80,0.40,0.70,0.6,0.60,0.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19118.0,0.70,0.6,0.50,0.40,0.10,0.60,0.1,0.50,0.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19117.0,0.95,0.9,0.70,0.60,0.40,0.10,0.3,0.80,0.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,4600.0,0.95,0.9,0.75,0.50,0.60,0.20,0.1,0.30,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
573,4613.0,0.95,0.9,0.70,0.30,0.60,0.05,0.1,0.50,0.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
574,4612.0,0.80,0.7,0.90,0.20,0.30,0.10,0.6,0.70,0.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# prompt: write code to save df_results to comma seperate file as "midsprojects_gemini_flash_results.csv" in gs://prabhutest/ storage location

df_results.to_csv("midsprojects_gemini_flash_results.csv", index=False)
!gsutil cp midsprojects_gemini_flash_results.csv gs://prabhutest/

Copying file://midsprojects_gemini_flash_results.csv [Content-Type=text/csv]...
/ [1 files][ 38.9 KiB/ 38.9 KiB]                                                
Operation completed over 1 objects/38.9 KiB.                                     


In [ ]:
# prompt: write code to join df_results and data pandas dataframes on a column "ID" and write resulting dataframe to file gs://prabhutest/

merged_df = pd.merge(data, df_results, on='ID', how='left')
merged_df.to_csv("midsprojects_data_final_geminiflash.csv", sep='\t',index=False)
!gsutil cp midsprojects_data_final_geminiflash.csv gs://prabhutest/

Copying file://midsprojects_data_final_geminiflash.csv [Content-Type=text/csv]...
/ [1 files][  1.4 MiB/  1.4 MiB]                                                
Operation completed over 1 objects/1.4 MiB.                                      
